In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt


In [4]:
# XGBOOST, feature importances
print(data["EASEMENT"].unique())

[nan 'E' 'F' 'G' 'H' 'N' 'I']


In [5]:
# GEOGRAFICZNIE UZUPELNIAMY NA PODSTAWIE BLOKÓW
# geograficzne to:staddr, postcode, latitude, longitude, community board
# council district, census tract, NTA
# STORIES - BIERZEMY SREDNIA Z BLOKOW
# ext na dwie kolumny e i g
# EASEMENT - DWIE KOLUMNY E I N
# obliczyc powierzchnie i normalizować

In [32]:
def prepare_data(path):
    data = pd.read_csv(path)
    data.drop(labels=["BBLE", "LOT","PERIOD", "VALTYPE",
                  "Borough", "New Georeferenced Column",
                  "OWNER", "AVLAND2", "AVTOT2",
                  "EXLAND2", "EXTOT2","BIN", "EXCD1", "EXCD2",
                  "EXMPTCL", "YEAR"],axis=1, inplace=True)
    data["EASEMENT"] = data["EASEMENT"].fillna("0")
    data["EXT"] = data["EXT"].fillna("0")
    data.loc[(data["EASEMENT"] == "F"), "EASEMENT"] = "E"
    data.loc[(data["EASEMENT"] == "G"), "EASEMENT"] = "E"
    data.loc[(data["EASEMENT"] == "H"), "EASEMENT"] = "E"
    data.loc[(data["EASEMENT"] == "I"), "EASEMENT"] = "E"
    data["EXT_E"] = 0
    data["EXT_G"] = 0
    data["EASEMENT_E"] = 0
    data["EASEMENT_N"] = 0
    data.loc[(data["EXT"] == "E"), "EXT_E"] = 1
    data.loc[(data["EXT"] == "EG"), "EXT_E"] = 1
    data.loc[(data["EXT"] == "G"), "EXT_G"] = 1
    data.loc[(data["EXT"] == "EG"), "EXT_G"] = 1
    data.loc[(data["EASEMENT"] == "E"), "EASEMENT_E"] = 1
    data.loc[(data["EASEMENT"] == "N"), "EASEMENT_N"] = 1
    data.drop(["EXT", "EASEMENT"], axis=1, inplace=True)
    data["STORIES"].fillna(method="ffill", inplace=True)
    data['LTSURF'] = data.apply(lambda row: row.LTFRONT * row.LTDEPTH, axis=1)
    return data

In [33]:
data = prepare_data("data/train_data.csv")

In [34]:
round(data.isnull().sum()/len(data), 5)

BORO                0.00000
BLOCK               0.00000
BLDGCL              0.00000
TAXCLASS            0.00000
LTFRONT             0.00000
LTDEPTH             0.00000
STORIES             0.00000
FULLVAL             0.00000
AVLAND              0.00000
AVTOT               0.00000
EXLAND              0.00000
EXTOT               0.00000
STADDR              0.00142
POSTCODE            0.02947
BLDFRONT            0.00000
BLDDEPTH            0.00000
Latitude            0.03633
Longitude           0.03633
Community Board     0.03633
Council District    0.03633
Census Tract        0.03633
NTA                 0.03633
EXT_E               0.00000
EXT_G               0.00000
EASEMENT_E          0.00000
EASEMENT_N          0.00000
LTSURF              0.00000
dtype: float64